In [ ]:
COLAB_FLAG = False   # whether running on colab or locally on computer

In [ ]:
if COLAB_FLAG:
    !pip install pydicom==2.1.2
    !pip install monai seaborn sentence_transformers
    !git clone 'https://github.com/mille055/AIPI540_individual_project.git'

    


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import torchvision
import pydicom
#import monai
import pickle
import glob
import sklearn
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression

import seaborn as sns
from datetime import datetime
from sentence_transformers import SentenceTransformer

from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import time
import os
import copy
from pprint import pprint
from fastai.basics import delegates
from fastcore.parallel import parallel
from fastcore.utils import gt
from fastcore.foundation import L

# import monai
# from monai.data import DataLoader, ImageDataset
# from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, EnsureType
from pydicom.dataset import Dataset as DcmDataset
from pydicom.tag import BaseTag as DcmTag
from pydicom.multival import MultiValue as DcmMultiValue
import sys
import importlib
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#local imports
if COLAB_FLAG:
    sys.path.append('/content/AIPI540_individual_project/scripts/')
    train_datafile = '/content/AIPI540_individual_project/data/trainfiles.csv'
    val_datafile = '/content/AIPI540_individual_project/data/valfiles.csv'
    test_datafile = '/content/AIPI540_individual_project/data/testfiles.csv'

else: # running locally
    sys.path.append('/Users/cmm/Documents/GitHub/AIPI540_individual_project/scripts/')
    train_datafile = '../data/trainfiles.csv'
    val_datafile = '../data/valfiles.csv'
    test_datafile = '../data/testfiles.csv'

### local imports ###
from config import file_dict, feats, feats_to_keep, column_lists, RF_parameters, classes, model_paths
from config import abd_label_dict, val_list, train_val_split_percent, random_seed, data_transforms
from config import sentence_encoder, series_description_column
from utils import *
#from train_pixel_model import train_pix_model, test_pix_model, ImgDataset, image_to_tensor, get_pixel_preds_and_probs
#from train_meta_model import train_fit_parameter_trial, train_meta_model, calc_feature_importances, get_meta_probs, meta_inference
#from train_text_model import train_text_log_model, load_text_data, get_nlp_inference
from NLP.NLP_inference import get_NLP_inference
from NLP.NLP_training import train_NLP_model
from cnn.cnn_dataset import ImgDataset
from cnn.cnn_inference import image_to_tensor, pixel_inference, test_pix_model, load_pixel_model, visualize_results
from cnn.cnn_model import CustomResNet50
from cnn.cnn_data_loaders import get_data_loaders
from metadata.meta_inference import meta_inference, calc_feature_importances
from metadata.meta_training import train_fit_parameter_trial, train_meta_model, evaluate_meta_model
from fusion_model.fus_model import FusionModel
from fusion_model.fus_inference import get_fusion_inference
from model_container import ModelContainer
from process_tree import Processor, write_labels_into_dicom
# from AIPI540_individual_project.scripts.train_pixel_model import train_model
# from AIPI540_individual_project.scripts.train_text_model import load_text_data, train_text_model, list_incorrect_text_predictions
# from AIPI540_individual_project.scripts.utils import *

In [ ]:
# ### fix my fusion model issue:; have to change from a pickled model to saved weights
# def save_fusion_model_weights(pickle_path, weight_path):
#     with open(pickle_path, 'rb') as file:
#         fusion_model = pickle.load(file)
#     torch.save(fusion_model.state_dict(), weight_path)

# save_fusion_model_weights(model_paths['fusion'], '../models/fusion_saved_weights042123.pth')
# save_fusion_model_weights('../models/meta_and_pixel_fusion_model041623.pkl', '../models/fusion_saved_weights_no_nlp042123.pth')

In [ ]:
### in the fusion model creation code:

Xtrain1 = fusion_train_df.meta_probs.values

Xtrain2 = fusion_train_df.pixel_probs.values
Xtrain3 = fusion_train_df.nlp_probs.values

Xtrain1 = np.stack(Xtrain1, axis=0)
Xtrain2 = np.stack(Xtrain2, axis=0)
Xtrain3 = np.stack(Xtrain3, axis=0)

out_features = 19


y_train_fusion = np.array([classes.index(x) for x in fusion_train_df.true])


In [ ]:
# pickling the dataframes which have undergone inference from the base models

# fusion_train_df.to_pickle('../data/fusion_train.pkl')
# fusion_val_df.to_pickle('../data/fusion_val.pkl')
# fusion_test_df.to_pickle('../data/fusion_test.pkl')


In [ ]:
# class FusionModel(nn.Module):
#     def __init__(self, model1, model2, model3, num_classes):
#         super(FusionModel, self).__init__()
#         self.model1 = model1
#         self.model2 = model2
#         self.model3 = model3
#         self.fusion_layer = nn.Linear(num_classes * 3, num_classes)

#     def forward(self, x1, x2, x3):
        
#         x = torch.cat((x1, x2, x3), dim=0)
#         x = self.fusion_layer(x)

#         return x

In [ ]:
class PartialFusionModel(nn.Module):
    def __init__(self, model1, model2, num_classes):
        super(PartialFusionModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        #self.model3 = model3
        self.fusion_layer = nn.Linear(num_classes * 2, num_classes)

    def forward(self, x1, x2):
        
        x = torch.cat((x1, x2), dim=0)
        x = self.fusion_layer(x)

        return x
    

# ## since I merged this into FusionModel with conditions after saving the weights, i need to try to recover the weights into 
# ## the new mdoel
# # load components


# #pmodel = PartialFusionModel(model_pix, meta_model, num_classes=19)

# from config import model_paths
# model_container = ModelContainer()

# with open(model_paths['fusion_no_nlp'], 'rb') as file:
#     partial_fusion_model = pickle.load(file)

# fusion_model_with_partial_weights = FusionModel(models, num_classes=19, include_nlp=False)
# fusion_model_with_partial_weights.fusion_layer.weight = nn.Parameter(partial_fusion_model.fusion_layer.weight)

# # Save only the model weights (state_dict)
# torch.save(fusion_model_with_partial_weights.state_dict(), '../models/fusion_model_no_nlp042123.pth')

